In [2]:
from CB_02704 import *
path = "../02704_data"
import numpy as np
import matplotlib.pyplot as plt

In [3]:
ceu_geno = read_geno_pop("ceu")
yri_geno = read_geno_pop("yri")

ceu_geno_ch1 = np.ma.filled(ceu_geno, 0)
yri_geno_ch1 = np.ma.filled(yri_geno, 0)


ceu_snp = read_snp(f"{path}/ceu.snp")
yri_snp = read_snp(f"{path}/yri.snp")

In [ ]:
asw_geno = read_geno_pop("asw")
asw_geno = np.ma.filled(asw_geno, 0)
asw_snp = read_snp(f"{path}/asw.snp")

In [ ]:
np.save("geno_files/ceu.npy", ceu_geno)
np.save("geno_files/yri.npy", yri_geno)
np.save("geno_files/asw.npy", asw_geno)

In [ ]:
# ceu_geno_ch1 = read_geno_pop_chr("ceu", 1)
# yri_geno_ch1 = read_geno_pop_chr("yri", 1)

# ceu_geno_ch1 = np.ma.filled(ceu_geno_ch1, 0)
# yri_geno_ch1 = np.ma.filled(yri_geno_ch1, 0)

# ceu_snp = read_snp(f"{path}/ceu.snp")
# yri_snp = read_snp(f"{path}/yri.snp")

# ceu_snp.index = range(0, len(ceu_snp))
# yri_snp.index = range(0, len(yri_snp))

# ceu_snp_ch1 = ceu_snp[ceu_snp["chromosome"] == 1]
# yri_snp_ch1 = yri_snp[yri_snp["chromosome"] == 1]

In [ ]:
print(ceu_snp)

        chromosome  morgans   position ref alt
0                1      0.0     742584   G   A
1                1      0.0     744045   G   A
2                1      0.0     744197   C   T
3                1      0.0     750775   T   C
4                1      0.0     758311   G   A
...            ...      ...        ...  ..  ..
718843          23      0.0  154569169   G   A
718844          23      0.0  154570039   G   T
718845          23      0.0  154578239   C   T
718846          23      0.0  154580775   A   G
718847          23      0.0  154582606   C   T

[718848 rows x 5 columns]


In [ ]:
# use poisson random variable to represent the number of recombination events that occur along the genome
# genome_end - genome_start represents the range of possible locations for recombination events
# then we sample n_recombination_events uniform random variables to represent each site of recombination

def simulate_offspring_genomes(n_offspring, pop1_geno, pop2_geno, snp):
    N1 = pop1_geno.shape[1]
    N2 = pop2_geno.shape[1]

    
    start_pos = 0
    end_pos = len(pop1_geno)

    offspring_genomes = []
    current_genome = np.random.randint(2)
    for i in range(n_offspring):
        
        genome_start = snp["position"].iloc[0]
        genome_end = snp["position"].iloc[-1]

        lambda_ = 0.000001 * (genome_end - genome_start) # adjust n_recombination events

        n_recombination_events = np.random.poisson(lam = lambda_, size=1)
        recombination_sites = np.array(np.floor(np.random.uniform(genome_start, genome_end, n_recombination_events)) % len(pop1_geno), dtype=int)
        recombination_sites = sorted(recombination_sites)
        
        
        
        offspring_genome = np.array([])
        
        mate1 = np.random.randint(0, N1)
        mate2 = np.random.randint(0, N2)
        
        
        current_pos = start_pos
        
        for site in recombination_sites:
            if current_genome == 0:
                offspring_genome = np.append(offspring_genome, pop1_geno[current_pos:site, mate1])
                current_pos = site
                current_genome = 1
            elif current_genome == 1:
                offspring_genome = np.append(offspring_genome, pop2_geno[current_pos:site, mate2])
                current_pos = site
                current_genome = 0
        if current_genome == 0:
            offspring_genome = np.append(offspring_genome, pop1_geno[site:len(pop1_geno), mate1])
        elif current_genome == 1:
            offspring_genome = np.append(offspring_genome, pop2_geno[site:len(pop2_geno), mate2])
        offspring_genomes.append(offspring_genome)
    return offspring_genomes

simulated_genomes = simulate_offspring_genomes(150, ceu_geno, yri_geno, ceu_snp)
simulated_genomes = np.array(simulated_genomes, dtype=np.int).T
print(simulated_genomes.shape)

(718848, 150)


In [ ]:
np.save("geno_files/sim_geno.npy", simulated_genomes)

(718848, 112)
(718848, 113)
(718848, 150)


: 

In [ ]:
def calc_allele_freq(geno):
    return np.sum(geno) / (2*len(geno))


def two_pop_allele_freqs(pop1_geno, pop2_geno):
    freq_array = np.empty(2, dtype=int)
    
    for i in range(len(pop1_geno)):
        pop1_freq = calc_allele_freq(pop1_geno[i])
        pop2_freq = calc_allele_freq(pop2_geno[i])
        
        freq_array = np.vstack((freq_array, [pop1_freq, pop2_freq]))
    freq_array = freq_array[1:]
    return freq_array

def compute_snp_pops(sim_pop_geno, freq_array):
    snp_belongs_to = []
    for i in range(len(sim_pop_geno)):
        sim_pop_freq = calc_allele_freq(sim_pop_geno[i])
        if sim_pop_freq > 0.5:
            snp_belongs_to.append(np.argmax(freq_array[i]))
        else:
            snp_belongs_to.append(np.argmin(freq_array[i]))
    return snp_belongs_to


In [ ]:

freq_array = two_pop_allele_freqs(ceu_geno_ch1, yri_geno_ch1)

simulated_snp_pops = compute_snp_pops(simulated_genomes, freq_array)
simulated_snp_pops = sorted(simulated_snp_pops)

[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 